__Purpose:__ Introduce Federated Learning, specifically by implementing FedAveraging on our dataset and moving on to more advanced methods.
<br>
1. Need to figure out what the weights are
2. Need to look into asynchronous FL
3. Create a global decoder from the last update... can I test it? I don't think so...
4. I still think it could be beneficial to create a random external model that we could easily do FL on...
5. Does scipy.optimize.minimize() run 1 iter or all necessary? Can it be replaced with SGD?  How are BFGS and SGD related?

In [1]:
#import h5py as h5
#import aopy 
import pandas as pd
import os
import numpy as np
#from numpy.matlib import repmat
#from matplotlib import pyplot as plt
#from scipy.signal import detrend, firwin, freqz, lfilter
#from sklearn.model_selection import train_test_split, ShuffleSplit
from scipy.optimize import minimize, least_squares
import copy
from itertools import permutations

In [2]:
from experiment_params import *
from simulations import *
import time
import pickle5 as pickle

## Load Our Data In

In [3]:
emg_labels_df1 = pd.read_csv("Data\emg_full_labels1.csv")
#emg_labels_df2 = pd.read_csv("Data\emg_full_labels2.csv")
emg_labels_df = emg_labels_df1
#emg_labels_df = pd.concat((emg_labels_df1, emg_labels_df2))

try:
    emg_labels_df.drop('Unnamed: 0', axis=1, inplace=True)
except:
    # Masterful code here
    print("NO UNNAMED COLUMN DETECTED!")
    
print(emg_labels_df["Subject"].unique())
print()
print(emg_labels_df.shape[0]/14/64/8)
# 14 participants / 64 channels / 8 conditions
# Why do I only have 2 updates...

print(emg_labels_df.shape)
display(emg_labels_df.head())

labels_df = pd.DataFrame(emg_labels_df['Subject'].map(key_to_num))
labels_df.head()

['METACPHS_S106' 'METACPHS_S107' 'METACPHS_S108' 'METACPHS_S109'
 'METACPHS_S110' 'METACPHS_S111' 'METACPHS_S112' 'METACPHS_S113'
 'METACPHS_S114' 'METACPHS_S115' 'METACPHS_S116' 'METACPHS_S117'
 'METACPHS_S118' 'METACPHS_S119']

1.0
(7168, 3)


,Subject,Condition,Channel
0,METACPHS_S106,0,0
1,METACPHS_S106,0,1
2,METACPHS_S106,0,2
3,METACPHS_S106,0,3
4,METACPHS_S106,0,4


,Subject
0,0
1,0
2,0
3,0
4,0


In [4]:
'''
t0 = time.time()
emg_data_df1 = pd.read_csv("Data\emg_full_data1.csv")
emg_data_df2 = pd.read_csv("Data\emg_full_data2.csv")
emg_data_df = pd.concat((emg_data_df1, emg_data_df2))
try:
    emg_data_df.drop('Unnamed: 0', axis=1, inplace=True)
except:
    print("NO UNNAMED COLUMN DETECTED!")
t1 = time.time()
total = t1-t0  
print(total)
print(emg_data_df.shape)
emg_data_df.head()
'''
# Just use the emg data directly from the pickle file for now
;

''

In [5]:
'''
t0 = time.time()
#envelope_df50 = pd.read_csv("Data\envelope_df50.csv")
envelope_df100 = pd.read_csv("Data\envelope_df100.csv")
#envelope_df150 = pd.read_csv("Data\envelope_df150.csv")
#envelope_df200 = pd.read_csv("Data\envelope_df200.csv")
#envelope_df250 = pd.read_csv("Data\envelope_df250.csv")
#envelope_df300 = pd.read_csv("Data\envelope_df300.csv")
#raw_envs = [envelope_df50, envelope_df100, envelope_df150, envelope_df200, envelope_df250, envelope_df300]
#all_envs = [env.drop('Unnamed: 0', axis=1) for env in raw_envs]
try:
    envelope_df100.drop('Unnamed: 0', axis=1, inplace=True)
except:
    print("NO UNNAMED COLUMN DETECTED!")
t1 = time.time()
total = t1-t0  
print(total)
print(envelope_df100.shape)
envelope_df100.head()
'''
# Just use the emg data directly from the pickle file for now
;

''

In [6]:
t0 = time.time()

with open('Data\continuous_full_data_block1.pickle', 'rb') as handle:
    #refs_block1, poss_block1, dec_vels_block1, int_vel_block1, emgs_block1, Ws_block1, Hs_block1, alphas_block1, pDs_block1, times_block1, conditions_block1 = pickle.load(handle)
    refs_block1, _, _, _, emgs_block1, Ws_block1, _, _, _, _, _ = pickle.load(handle)

#with open('Data\continuous_full_data_block2.pickle', 'rb') as handle:
    #refs_block2, poss_block2, dec_vels_block2, int_vel_block2, emgs_block2, Ws_block2, Hs_block2, alphas_block2, pDs_block2, times_block2, conditions_block2 = pickle.load(handle)
    #refs_block2, _, _, _, emgs_block2, Ws_block2, _, _, _, _, _ = pickle.load(handle)

t1 = time.time()
total = t1-t0  
print(total)

2.4497461318969727


In [7]:
# 8 conditions, 20770 data points (only 19 unique sets!), xy, channels
Ws_block1[keys[0]].shape

(8, 20770, 2, 64)

In [8]:
update_ix

array([    0,  1200,  2402,  3604,  4806,  6008,  7210,  8412,  9614,
       10816, 12018, 13220, 14422, 15624, 16826, 18028, 19230, 20432,
       20769])

In [9]:
dec_cond0_user1_update0 = Ws_block1[keys[0]][0,0,:,:]
dec_cond0_user1_update1 = Ws_block1[keys[0]][0,update_ix[1],:,:]
dec_cond0_user1_update2 = Ws_block1[keys[0]][0,update_ix[2],:,:]

print(f"Shape of decoder: {dec_cond0_user1_update0.shape}")
print()
print(f"Total difference between dec0 and dec1: {(dec_cond0_user1_update0 - dec_cond0_user1_update1).sum()}")
print("E.g., as previously shown, the first two decs are the same")
print()
print(f"Total difference between dec0 and dec2: {(dec_cond0_user1_update0 - dec_cond0_user1_update2).sum()}")

Shape of decoder: (2, 64)

Total difference between dec0 and dec1: 0.0
E.g., as previously shown, the first two decs are the same

Total difference between dec0 and dec2: 3.1981579823181594


In [10]:
#emg_cond0_user1_update0 = emg_data_df.iloc[:64,:].shape

## Run One Iteration On Above Data and Check Decoders Are the Same
1. Modifying Simulations Code

In [11]:
# Just 1 person
filtered_signals = emgs_block1[keys[0]][0,:,:]
# Read in the reference positions from the pickle file
cued_target_position = refs_block1[keys[0]][0,:,:]

In [12]:
# Previously created random decoder, but we are trying to rerun
#D_0 = np.random.rand(2,64)
D_0 = dec_cond0_user1_update0

#learning_batch = 8
learning_batch = update_ix[1]  # I think this is supposed to be the number of datapoints per update?... why was it only 8 before then? Still don't know where they were getting 60 data points from
num_dps_per_update = update_ix[1]  #1200

In [13]:
alpha = .95 # higher alpha means more old decoder (slower update)
alphaF=1e-1
alphaD = 1e-1

D = []
D_constant = []
D_bounded = []
D_constant_bounded = []
D.append(D_0)
D_constant.append(D_0)
D_bounded.append(D_0)
D_constant_bounded.append(D_0)
accuracy = []
accuracy_constant = []
accuracy_bounded = []
accuracy_constant_bounded = []

In [14]:
# Original code for running simulations...
# for ix in range(10000):
    #accuracy_constant_,D_constant,p_constrained_constant = simulation_constant_intent(D_constant,learning_batch,alpha,alphaF=alphaF,alphaD=alphaD)
    #accuracy_constant.extend(accuracy_constant_)
    #accuracy_,D,p_constrained = simulation(D,learning_batch,alpha,alphaF=alphaF,alphaD=alphaD)    
    #accuracy.extend(accuracy_)
    #accuracy_bounded_,D_bounded,p_bounded = simulation_bounded_pos(D_bounded,learning_batch,alpha,alphaF=alphaF,alphaD=alphaD)  
    #accuracy_bounded.extend(accuracy_bounded_)
    #accuracy_constant_bounded_,D_constant_bounded,p_constant_bounded = simulation_constant_intent_bounded(D_constant_bounded,learning_batch,alpha,alphaF=alphaF,alphaD=alphaD)
    #accuracy_constant_bounded.extend(accuracy_constant_bounded_)
    
# Modified code for running simulations...
# Why loop at all right now...
#for ix in range(10):
    #accuracy_,D,p_constrained = simulation(D,learning_batch,alpha,alphaF=alphaF,alphaD=alphaD)    


In [15]:
# Added 2 new parameters
#def simulation(D,learning_batch,alpha,alphaF=1e-2,alphaD=1e-2,display_info=False,num_iters=False):
#D  # Already defined
#learning_batch  # Already defined
#alpha  # Already defined
#alphaF=1e-2  #defined as something else earlier...
#alphaD=1e-2  #defined as something else earlier...
display_info=True
num_iters=False

In [37]:
p_classify = []
accuracy_temp = []

#num_updates = int(np.floor((filtered_signals.shape[0]-1)/learning_batch)) # how many times can we update decoder based on learning batch    
num_updates = 19  # Let this equal the number of actual updates

#############################################################################################
# RANDOMIZE DATASET
# Idk what's going on here
randomized_integers = np.random.permutation(range(0,cued_target_position.shape[0]))
filtered_signals_randomized = filtered_signals  # filtered_signals[randomized_integers]
cued_target_position_randomized = cued_target_position  # cued_target_position[randomized_integers]
#############################################################################################

# batches the trials into each of the update batch
for ix in range(num_updates):
    #s = np.hstack([x for x in filtered_signals_randomized[int(ix*learning_batch+1):int((ix+1)*learning_batch+1),:,:]])# stack s (64 x (60 timepoints x learning batch size))
    # stack s (64 x (60 timepoints x learning batch size))
    #s = filtered_signals_randomized[:, int(ix*learning_batch+1):int((ix+1)*learning_batch+1)]
    # Idk why they had all these +1s...
    s = np.transpose(filtered_signals_randomized[:, int(ix*learning_batch):int((ix+1)*learning_batch+1)])
    #p_intended = np.hstack([np.tile(x[:,np.newaxis],60) for x in cued_target_position_randomized[int(ix*learning_batch+1):int((ix+1)*learning_batch+1),:]]) # stack p_intended (2 x 60 timepoints x learning batch size)
    p_intended = np.hstack([np.tile(x[:,np.newaxis],learning_batch) for x in cued_target_position_randomized[int(ix*learning_batch+1):int((ix+1)*learning_batch+1),:]]) # stack p_intended (2 x 60 timepoints x learning batch size)
    # Band-aid solution
    p_intended = p_intended[:, :20770]
    v_intended,p_constrained = output_new_decoder(s,D[-1],p_intended)

    # CLASSIFY CURRENT DECODER ACCURACY
    v_actual = D[-1]@s
    for trial in range(learning_batch):
        v_trial = v_actual[:,int(trial*num_dps_per_update):int((trial+1)*num_dps_per_update)] # velocities for each trials (2,60)
        p_final = np.sum(v_trial,axis=1)[:,np.newaxis] # final position after integration (2,)
        p_classify.append(classify(p_final, cued_target_position))

    # UPDATE DECODER
    u = copy.deepcopy(s) # u is the person's signal s (64 CHANNELS X TIMEPOINTS)
    q = copy.deepcopy(v_intended) # use cued positions as velocity vectors for updating decoder should be 2 x num_trials

    # emg_windows against intended_targets (trial specific cued target)
    F = copy.deepcopy(u[:,:-1]) # note: truncate F for estimate_decoder
    V = copy.deepcopy(q)

    # initial decoder estimate for gradient descent
    D0 = np.random.rand(2,64)
    # Why is this different from D_0?

    # set alphas
    H = np.zeros((2,2))
    # use scipy minimize for gradient descent and provide pre-computed analytical gradient for speed
    if num_iters is False:
        out = minimize(lambda D: cost_l2(F,D,H,V), D0, method='BFGS', jac = lambda D: gradient_cost_l2(F,D,H,V), options={'disp': display_info})
    else:
        out = minimize(lambda D: cost_l2(F,D,H,V), D0, method='BFGS', jac = lambda D: gradient_cost_l2(F,D,H,V), options={'disp': display_info, 'maxiter':num_iters})

    # reshape to decoder parameters
    W_hat = np.reshape(out.x,(2, 64))

    # DO SMOOTHBATCH
    W_new = alpha*D[-1] + ((1 - alpha) * W_hat)
    D.append(W_new)

    # COMPUTE CLASSIFICATION ACURACY 
    # This is definitely wrong, not sure what it should be changed to...
    p_target = (cued_target_position[randomized_integers])[int(ix*learning_batch+1):int((ix+1)*learning_batch+1),:] # obtain target
    accuracy_temp.append(classification_accuracy(p_target,p_classify[-learning_batch:]))

p_classify = np.asarray(p_classify)
#return accuracy_temp,D,p_constrained

Optimization terminated successfully.
         Current function value: 21935.005158
         Iterations: 140
         Function evaluations: 188
         Gradient evaluations: 188


ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 0 is different from 64)

In [36]:
s.shape

(1201, 64)

In [32]:
ix

1

In [31]:
len(D)

2

In [17]:
D[-1].shape

(2, 64)

In [30]:
s.shape

(0, 20770)

In [28]:
filtered_signals.shape

(20770, 64)

In [29]:
filtered_signals_randomized.shape

(20770, 64)

In [21]:
np.transpose(filtered_signals_randomized[:, int(ix*learning_batch):int((ix+1)*learning_batch+1)]).shape

(0, 20770)

In [23]:
int(ix*learning_batch)

1200

In [24]:
int((ix+1)*learning_batch+1)

2401

In [27]:
filtered_signals_randomized[:, 1200:2401].shape

(20770, 0)

In [33]:
filtered_signals.shape

(20770, 64)

In [61]:
s.shape[1]

1200

In [62]:
p_classify = []
accuracy_temp = []

#num_updates = int(np.floor((filtered_signals.shape[0]-1)/learning_batch)) # how many times can we update decoder based on learning batch    
num_updates = 19  # Let this equal the number of actual updates

#############################################################################################
# RANDOMIZE DATASET
# Idk what's going on here
randomized_integers = np.random.permutation(range(0,cued_target_position.shape[0]))
filtered_signals_randomized = np.transpose(filtered_signals)  # filtered_signals[randomized_integers]
print(filtered_signals_randomized.shape)
cued_target_position_randomized = cued_target_position  # cued_target_position[randomized_integers]
print(cued_target_position_randomized.shape)
#############################################################################################

# batches the trials into each of the update batch
for ix in range(num_updates):
    #s = np.hstack([x for x in filtered_signals_randomized[int(ix*learning_batch+1):int((ix+1)*learning_batch+1),:,:]])# stack s (64 x (60 timepoints x learning batch size))
    # stack s (64 x (60 timepoints x learning batch size))
    #s = filtered_signals_randomized[:, int(ix*learning_batch+1):int((ix+1)*learning_batch+1)]
    # Idk why they had all these +1s...
    s = filtered_signals_randomized[:, int(ix*learning_batch):int((ix+1)*learning_batch)]
    #p_intended = np.hstack([np.tile(x[:,np.newaxis],60) for x in cued_target_position_randomized[int(ix*learning_batch+1):int((ix+1)*learning_batch+1),:]]) # stack p_intended (2 x 60 timepoints x learning batch size)
    p_intended = np.hstack([np.tile(x[:,np.newaxis],learning_batch) for x in cued_target_position_randomized[int(ix*learning_batch+1):int((ix+1)*learning_batch+1),:]]) # stack p_intended (2 x 60 timepoints x learning batch size)
    # Band-aid solution
    p_intended = p_intended[:, :s.shape[1]]
    v_intended,p_constrained = output_new_decoder(s,D[-1],p_intended)

    # CLASSIFY CURRENT DECODER ACCURACY
    v_actual = D[-1]@s
    for trial in range(learning_batch):
        v_trial = v_actual[:,int(trial*num_dps_per_update):int((trial+1)*num_dps_per_update)] # velocities for each trials (2,60)
        p_final = np.sum(v_trial,axis=1)[:,np.newaxis] # final position after integration (2,)
        p_classify.append(classify(p_final, cued_target_position))

    # UPDATE DECODER
    u = copy.deepcopy(s) # u is the person's signal s (64 CHANNELS X TIMEPOINTS)
    q = copy.deepcopy(v_intended) # use cued positions as velocity vectors for updating decoder should be 2 x num_trials

    # emg_windows against intended_targets (trial specific cued target)
    F = copy.deepcopy(u[:,:-1]) # note: truncate F for estimate_decoder
    V = copy.deepcopy(q)

    # initial decoder estimate for gradient descent
    D0 = np.random.rand(2,64)
    # Why is this different from D_0?

    # set alphas
    H = np.zeros((2,2))
    # use scipy minimize for gradient descent and provide pre-computed analytical gradient for speed
    if num_iters is False:
        out = minimize(lambda D: cost_l2(F,D,H,V), D0, method='BFGS', jac = lambda D: gradient_cost_l2(F,D,H,V), options={'disp': display_info})
    else:
        out = minimize(lambda D: cost_l2(F,D,H,V), D0, method='BFGS', jac = lambda D: gradient_cost_l2(F,D,H,V), options={'disp': display_info, 'maxiter':num_iters})

    # reshape to decoder parameters
    W_hat = np.reshape(out.x,(2, 64))

    # DO SMOOTHBATCH
    W_new = alpha*D[-1] + ((1 - alpha) * W_hat)
    D.append(W_new)

    # COMPUTE CLASSIFICATION ACURACY 
    # This is definitely wrong, not sure what it should be changed to...
    p_target = cued_target_position[-learning_batch:, :]  # (cued_target_position[randomized_integers])[int(ix*learning_batch+1):int((ix+1)*learning_batch+1),:] # obtain target
    accuracy_temp.append(classification_accuracy(p_target,p_classify[-learning_batch:]))

p_classify = np.asarray(p_classify)
#return accuracy_temp,D,p_constrained

(64, 20770)
(20770, 2)
Optimization terminated successfully.
         Current function value: 1106.912511
         Iterations: 148
         Function evaluations: 199
         Gradient evaluations: 199
Optimization terminated successfully.
         Current function value: 4708.949398
         Iterations: 149
         Function evaluations: 204
         Gradient evaluations: 204
Optimization terminated successfully.
         Current function value: 1213.727482
         Iterations: 149
         Function evaluations: 191
         Gradient evaluations: 191
Optimization terminated successfully.
         Current function value: 1619.567090
         Iterations: 150
         Function evaluations: 201
         Gradient evaluations: 201
Optimization terminated successfully.
         Current function value: 752.830458
         Iterations: 151
         Function evaluations: 195
         Gradient evaluations: 195
Optimization terminated successfully.
         Current function value: 1088.797473
     

ValueError: need at least one array to concatenate

In [58]:
s.shape

(64, 1200)

In [59]:
p_intended.shape

(2, 370)

In [60]:
D[-1].shape

(2, 64)

In [ ]:
print(D)